Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [135]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from copy import copy
from functools import reduce
from random import choice

In [136]:
# function that prints sets in a human readable way
def print_solution(solution, sets):
    
    for i, is_set_covered in enumerate(solution):
        if is_set_covered:
            print('Set', i+1, ': ', end='')
            for element_taken in sets.toarray()[i]:
                if(element_taken): print('x', end='')
                else: print('-', end='')
            print()


In [137]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [138]:
# x = make_set_covering_problem(1000, 1000, .3)
# print("Element at row=42 and column=42:", x[42, 42])

In [139]:
x1 = make_set_covering_problem(100, 100, .3)

In [140]:
x2 = make_set_covering_problem(1000, 1000, .3)

In [141]:
x3 = make_set_covering_problem(5000, 5000, .3)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Leonardo Pieraccioli\Documents\Esami\CI\computational-intelligence\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Leonardo Pieraccioli\AppData\Local\Temp\ipykernel_5516\1670192213.py", line 1, in <module>
    x3 = make_set_covering_problem(5000, 5000, .3)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Leonardo Pieraccioli\AppData\Local\Temp\ipykernel_5516\1353256190.py", line -1, in make_set_covering_problem
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Leonardo Pieraccioli\Documents\Esami\CI\computational-intelligence\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Us

In [ ]:
# x4 = make_set_covering_problem(100, 100, .7)
# x5 = make_set_covering_problem(1000, 1000, .7)
# x6 = make_set_covering_problem(5000, 5000, .7)

In [ ]:
def goal_test(sets, state):
    SETS = sets.toarray()
    return np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(len(SETS[0]))]),
        )
    )

In [ ]:
def fitness1(state, sets):
    SETS = sets.toarray()
    valid = np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(len(SETS[0]))]),
        )
    )
    return valid, sum(state)

In [ ]:
def tweak1(state, PROBLEM_SIZE, valid):
    new_state = copy(state)
    if valid:
        taken = []
        for i, t in enumerate(state):
            if t:
                taken.append(i)
        index = choice(taken)
    else: index = randint(0, PROBLEM_SIZE - 1)
    new_state[index] = not new_state[index]
    return new_state

In [ ]:
def single_state_solve(initial_state, tweak, fitness, sets, max_improvement=100,total_steps=1_000):
    cycles_needed = 0
    current_state = copy(initial_state)
    valid, current_fitness = fitness(current_state, sets)
    number_of_sets = len(initial_state)
    prev_valid = valid
    failed_improvement = 0
    
    for _ in range(total_steps):
        if failed_improvement >= max_improvement:
            break
        cycles_needed += 1
        new_state = tweak(current_state, number_of_sets, valid)
        valid, new_fitness = fitness(new_state, sets)
        if valid and new_fitness < current_fitness:
            print(f"Improvement at cycle: {cycles_needed}, ({valid}, {new_fitness})") 
            current_state = new_state
            current_fitness = new_fitness
        elif not valid and prev_valid:
            failed_improvement += 1
            valid = prev_valid
            continue
        elif not valid and not prev_valid and new_fitness > current_fitness:
            print(f"Still searching a valid solution. Current size {new_fitness}")
            current_state = new_state
            current_fitness = new_fitness
        prev_valid = valid

    return current_state, cycles_needed

In [142]:
initial_state_100 = [False for _ in range(100)]
solution1, cycles = single_state_solve(initial_state_100, tweak1, fitness1, x1, 10, 1000)
print(f"Solution has fitness: {fitness1(solution1, x1)} and was found in {cycles} cycles.")

Still searching a valid solution. Current size 1
Still searching a valid solution. Current size 2
Still searching a valid solution. Current size 3
Still searching a valid solution. Current size 4
Still searching a valid solution. Current size 5
Still searching a valid solution. Current size 6
Still searching a valid solution. Current size 7
Still searching a valid solution. Current size 8
Still searching a valid solution. Current size 9
Still searching a valid solution. Current size 10
Still searching a valid solution. Current size 11
Still searching a valid solution. Current size 12
Solution has fitness: (False, 12) and was found in 25 cycles.


In [143]:
initial_state_1000 = [False for _ in range(1000)]
solution2, cycles = single_state_solve(initial_state_1000, tweak1, fitness1, x2, 20, 1000)
print(f"Solution has fitness: {fitness1(solution2, x2)} and was found in {cycles} cycles.")

Still searching a valid solution. Current size 1
Still searching a valid solution. Current size 2
Still searching a valid solution. Current size 3
Still searching a valid solution. Current size 4
Still searching a valid solution. Current size 5
Still searching a valid solution. Current size 6
Still searching a valid solution. Current size 7
Still searching a valid solution. Current size 8
Still searching a valid solution. Current size 9
Still searching a valid solution. Current size 10
Still searching a valid solution. Current size 11
Still searching a valid solution. Current size 12
Still searching a valid solution. Current size 13
Still searching a valid solution. Current size 14
Still searching a valid solution. Current size 15
Still searching a valid solution. Current size 16
Still searching a valid solution. Current size 17
Still searching a valid solution. Current size 18
Still searching a valid solution. Current size 19
Still searching a valid solution. Current size 20
Still sea

In [144]:
initial_state_5000 = [False for _ in range(5000)]
solution3, cycles = single_state_solve(initial_state_5000, tweak1, fitness1, x3, 10, 1000)
print(f"Solution has fitness: {fitness1(solution3, x3)} and was found in {cycles} cycles.")

Still searching a valid solution. Current size 1
Still searching a valid solution. Current size 2
Still searching a valid solution. Current size 3
Still searching a valid solution. Current size 4
Still searching a valid solution. Current size 5
Still searching a valid solution. Current size 6
Still searching a valid solution. Current size 7
Still searching a valid solution. Current size 8
Still searching a valid solution. Current size 9
Still searching a valid solution. Current size 10
Still searching a valid solution. Current size 11
Still searching a valid solution. Current size 12
Still searching a valid solution. Current size 13
Still searching a valid solution. Current size 14
Still searching a valid solution. Current size 15
Still searching a valid solution. Current size 16
Still searching a valid solution. Current size 17
Still searching a valid solution. Current size 18
Still searching a valid solution. Current size 19
Still searching a valid solution. Current size 20
Still sea